# E. Weight Explorer (experimental)

A notebook to explore the influence of features on a neural network.

In [ ]:
# Disable tensorflow logging
import tensorflow as tf
import os
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [ ]:
# Specific instruction to run the notebooks from a sub-folder.
import sys
sys.path.append("..")

In [ ]:
import logging
from bugfinder.settings import LOGGER

from os.path import join, exists, basename, dirname
from shutil import rmtree, copytree

import tensorflow as tf
from pprint import pprint
from copy import deepcopy
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import xlsxwriter

In [ ]:
# Setup logging to only output INFO level messages
LOGGER.setLevel(logging.INFO)
LOGGER.propagate = False

In [ ]:
def prep_weights(input_weights, log=False):
    w_prep = list()
    
    for value in input_weights.values():
        last_col = value.columns[-1]
        w_prep.append(
            deepcopy(value.drop([last_col], axis=1).values.tolist())
        )
        
    return w_prep

def final_weights(input_weights, log=False):
    size = [len(layer[0]) for layer in input_weights]
    final_weights = np.zeros(size, dtype=np.float32)

    iterator = np.nditer(final_weights, flags=['multi_index'])
    for _ in iterator:
        idx = iterator.multi_index
        v = 1
        for n in range(len(idx) - 1):
            if log:
                print(f"Getting {n},{idx[n+1]},{idx[n]}: {input_weights[n].shape}")
            v *= input_weights[n][idx[n+1]][idx[n]]

        final_weights[idx] = v * input_weights[-1][0][idx[-1]]
        
    return final_weights

def revise_node_weights(node_weights, min_bound=0, max_bound=1):
    abs_values = np.abs(node_weights)
    min_value = np.min(abs_values)
    max_value = np.max(abs_values)
    
    if min_value == max_value:
        revised_node_weights = [0 for _ in abs_values]
    else:
        normalized_weights = [
            min_bound + (max_bound - min_bound) * (v - min_value) / (max_value - min_value) 
            for v in abs_values
        ]
        threshold = np.median(normalized_weights) - 0.5 * np.std(normalized_weights)

        revised_node_weights = [get_val(v, threshold) for v in normalized_weights]
    
    return revised_node_weights

def get_val(val, thr):
    return val if val >= thr else 0

def init_model(model_dir, model_arch, feat_count):
    # Renaming input columns to avoid forbidden characters
    columns = [
        f"feat{feature_nb}" for feature_nb in range(feat_count)
    ]

    model = tf.estimator.DNNClassifier(
        hidden_units = model_arch,
        feature_columns=[
            tf.feature_column.numeric_column(col) for col in columns
        ],
        n_classes=2,
        model_dir=model_dir,
    )

    return model, columns

In [ ]:
model_dir = "/home/pnd/model23"
model_arch = [10, 10, 10]
feat_count = 632
        
model, columns = init_model(
    model_dir, 
    model_arch, 
    feat_count
)

In [ ]:
chosen_model_vars = [var for var in model.get_variable_names() if basename(var) == "kernel"]
layers = sorted(set([basename(dirname(var)) for var in chosen_model_vars]))
cols = columns

weights = dict()

for layer in layers:
    kernel = model.get_variable_value(f"dnn/{layer}/kernel").transpose()
    
    layer_weights = np.zeros((kernel.shape[0], kernel.shape[1]+1))
    
    layer_weights[:, :-1] = kernel
    layer_weights[:, -1] = model.get_variable_value(f"dnn/{layer}/bias")
    
    weights[layer] = pd.DataFrame(layer_weights, columns=cols+["bias"])
    
    cols = [f"{layer}_n{index}" for index in range(layer_weights.shape[0])]

In [ ]:
# Step 1: correct layer value
new_weights = dict()

for layer in layers:
    layer_values = weights[layer].values
    new_layer_values = list()

    for node_values in layer_values:
        updated_node_values = revise_node_weights(node_values, -1, 1)
        new_layer_values.append(updated_node_values)
        
    new_weights[layer] = pd.DataFrame(
        new_layer_values, columns=[f"c{idx}" for idx in range(len(new_layer_values[0]))]
    )

In [ ]:
# Compute weights using orig weights
pww = prep_weights(weights)
fww = final_weights(pww)
fww_abs = np.abs(fww)

In [ ]:
# Compute weights using normalized weights
npw = prep_weights(new_weights)
nfw = final_weights(npw)
nfw_abs = np.abs(nfw)

In [ ]:
# Compute means
fww_avg = list()
fww_abs_avg = list()
nfw_avg = list()
nfw_abs_avg = list()

for idx in range(len(fww)):
    fww_avg.append(np.mean(fww[idx].flatten()))
    fww_abs_avg.append(np.mean(fww_abs[idx].flatten()))
    nfw_avg.append(np.mean(nfw[idx].flatten()))
    nfw_abs_avg.append(np.mean(nfw_abs[idx].flatten()))

In [ ]:
spread = 100
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
ax[0, 0].hist(fww_avg, spread, color='blue', alpha=0.75)
ax[0, 1].hist(fww_abs_avg, spread, color='orange', alpha=0.75)
ax[1, 0].hist(nfw_avg, spread, color='blue', alpha=0.75)
ax[1, 1].hist(nfw_abs_avg, spread, color='orange', alpha=0.75)
plt.show()

In [ ]:
thresholds = [0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10]
fw_corrs = [deepcopy(fww) for _ in range(len(thresholds))]

print(sum([np.count_nonzero(fw_corr.flatten()) for fw_corr in fw_corrs]) - len(fw_corrs[0].flatten())*len(fw_corrs))

print("Computing...")
for thr_idx, thr_val in tqdm(enumerate(thresholds), total=len(thresholds)):
    for fidx, fv in tqdm(enumerate(fww_abs), desc=f"Threshold: {thr_val}", total=len(fww_abs), leave=False):
        flat = fv.flatten()

        threshold = thr_val * np.std(flat)

        with np.nditer(fw_corrs[thr_idx][fidx], op_flags=['readwrite']) as it:
            for x in it:
                if abs(x) < threshold:
                    x[...] = 0.

print(sum([np.count_nonzero(fw_corr.flatten()) for fw_corr in fw_corrs]) - len(fw_corrs[0].flatten())*len(fw_corrs))

In [ ]:
fw_corrs_min = []
fw_corrs_max = []
fig, ax = plt.subplots(nrows=5, ncols=2, figsize=(20, 20))

fig_idx = 0
ax[int(np.floor(fig_idx/2)), fig_idx%2].hist(fww_avg, spread, alpha=0.75)

for fw_corr in fw_corrs:
    fw_corr_avg = []
    fig_idx += 1
    for idx in range(len(fww)):
        fw_corr_avg.append(np.mean(fw_corr[idx].flatten()))
        
    fw_corrs_min.append(np.min(fw_corr_avg))
    fw_corrs_max.append(np.max(fw_corr_avg))
    
    ax[int(np.floor(fig_idx/2)), fig_idx%2].hist(fw_corr_avg, spread, alpha=0.75)
    
plt.show()

In [ ]:
print(thresholds)
print(fw_corrs_min)
print(fw_corrs_max)

In [ ]:
chosen_corr = fw_corrs[3]
zeros = [(ft.size - np.count_nonzero(ft.flatten())) / ft.size for ft in chosen_corr]

plt.hist(zeros, spread, alpha=0.75)
plt.show()

In [ ]:
sorted_cols = np.argsort(zeros)
sorted_cols

In [ ]:
limited_cols = sorted_cols[-150:]
limited_cols

In [ ]:
aiwe = pd.read_csv("../data/aiwe-test-nopca3/sign01.csv")
aiwe_cols = np.array(aiwe.columns)

In [ ]:
limited_cols_for_pd = [aiwe_cols[idx] for idx in limited_cols]

In [ ]:
new_aiwe = aiwe.drop(aiwe.columns.difference(limited_cols_for_pd), axis=1)
new_aiwe.to_csv("../data/aiwe-test-nopca3/sign01-lim.csv", index=False)